In [1]:

# !pip install optuna
import optuna
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

c:\Users\Muhammad Mahdi\anaconda3\envs\prediksi-polusi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Membaca data dari file CSV bernama 'pollution_data.csv' ke dalam DataFrame df
df = pd.read_csv('pollution_data.csv')

# Mengubah kolom 'created_at' menjadi tipe data datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# Menghapus informasi zona waktu dari kolom 'created_at'
df['created_at'] = df['created_at'].dt.tz_localize(None)

# Menampilkan DataFrame df
df

,created_at,Temperature,Humidity,PM2.5,PM10,CO,CO2
0,2025-06-21 07:00:15,24.8,84.3,52.7,72.5,0.59860,403.34171
1,2025-06-21 07:01:04,24.8,84.1,50.4,66.7,0.56109,403.40472
2,2025-06-21 07:01:53,24.8,84.0,51.7,74.0,0.54466,403.36261
3,2025-06-21 07:02:43,24.9,83.6,49.7,65.9,0.55447,403.30020
4,2025-06-21 07:03:32,24.9,83.3,51.6,72.7,0.57113,403.29333
...,...,...,...,...,...,...,...
62420,2025-07-28 06:55:48,25.7,77.3,60.4,89.8,8.19941,411.28958
62421,2025-07-28 06:56:41,25.8,77.0,59.8,88.2,8.21437,411.35828
62422,2025-07-28 06:57:35,25.8,76.8,61.0,92.7,8.12498,411.18726
62423,2025-07-28 06:58:24,25.8,76.5,60.2,90.9,8.15468,411.41003


In [3]:
# Menghapus kolom 'Temperature' dan 'Humidity' dari DataFrame df
df = df.drop(columns=['Temperature', 'Humidity'])

# Membulatkan waktu pada kolom 'created_at' ke menit terdekat
df['created_at'] = df['created_at'].dt.floor('min')

# Memilih baris di mana menit pada 'created_at' adalah kelipatan 3
df = df[df['created_at'].dt.minute % 3 == 0]

# Menghapus duplikat berdasarkan 'created_at', hanya menyimpan yang pertama
df = df.drop_duplicates(subset='created_at', keep='first')

# Mengganti nama kolom 'created_at' menjadi 'datetime'
df = df.rename(columns={'created_at':'datetime'})

# Menjadikan kolom 'datetime' sebagai index DataFrame
df.set_index('datetime', inplace=True)

# Menambahkan kolom 'hour' yang berisi jam dari index
df['hour'] = df.index.hour

# Menambahkan kolom 'dayofweek' yang berisi hari dalam minggu dari index
df['dayofweek'] = df.index.dayofweek

# Menambahkan kolom 'minute' yang berisi menit dari index
df['minute'] = df.index.minute

# Menampilkan DataFrame df
df

,PM2.5,PM10,CO,CO2,hour,dayofweek,minute
datetime,,,,,,,
2025-06-21 07:00:00,52.7,72.5,0.59860,403.34171,7,5,0
2025-06-21 07:03:00,51.6,72.7,0.57113,403.29333,7,5,3
2025-06-21 07:06:00,50.2,67.4,0.55119,403.32782,7,5,6
2025-06-21 07:09:00,54.5,74.4,0.58302,403.24551,7,5,9
2025-06-21 07:12:00,55.0,72.6,0.53658,403.36960,7,5,12
...,...,...,...,...,...,...,...
2025-07-28 06:45:00,52.4,80.0,7.34412,410.49396,6,0,45
2025-07-28 06:48:00,53.6,84.1,7.20935,410.60675,6,0,48
2025-07-28 06:51:00,61.7,90.6,7.64829,410.65543,6,0,51


In [4]:
# Memilih fitur yang akan digunakan sebagai input (X) dan target (y)
features = ['PM2.5', 'PM10', 'CO', 'CO2']
features_y = ['PM2.5', 'PM10', 'CO', 'CO2']

# Mengambil data fitur dari DataFrame dan menghapus nilai yang hilang
data_X = df[features].dropna()
data_y = df[features_y].dropna()

# Membuat objek MinMaxScaler untuk normalisasi data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Melakukan normalisasi pada data fitur dan target
X_scaled = scaler_X.fit_transform(data_X)
y_scaled = scaler_y.fit_transform(data_y)

# Menampilkan bentuk (shape) dari data yang telah dinormalisasi
X_scaled.shape, y_scaled.shape

((17388, 4), (17388, 4))

In [5]:
joblib.dump(scaler_X, 'scaler_X.save')
joblib.dump(scaler_y, 'scaler_y.save')

['scaler_y.save']

In [6]:
# Fungsi create_sequences digunakan untuk membentuk data menjadi urutan (sequence) yang sesuai untuk model LSTM.
# X_data: data fitur yang sudah dinormalisasi
# y_data: data target yang sudah dinormalisasi
# n_steps_in: jumlah langkah waktu (timesteps) sebagai input
# n_steps_out: jumlah langkah waktu (timesteps) sebagai output

def create_sequences(X_data, y_data, n_steps_in, n_steps_out):
  X, y = [], []
  for i in range(len(X_data) - n_steps_in - n_steps_out + 1):
    # Mengambil urutan data sebagai input
    X.append(X_data[i:i+n_steps_in])
    # Mengambil urutan data sebagai target/output
    y.append(y_data[i+n_steps_in:i+n_steps_in+n_steps_out])
  return np.array(X), np.array(y)

# n_in: jumlah timesteps input
# n_out: jumlah timesteps output
n_in = 20
n_out = 20

# Membentuk data urutan untuk LSTM
X, y = create_sequences(X_scaled, y_scaled, n_in, n_out)

# Menampilkan bentuk (shape) dari data input dan output
X.shape, y.shape

((17349, 20, 4), (17349, 20, 4))

In [7]:
# Membagi data menjadi data latih dan data uji menggunakan train_test_split
# X_train, X_test: data fitur untuk pelatihan dan pengujian
# y_train, y_test: data target untuk pelatihan dan pengujian
# test_size=0.2: 20% data digunakan untuk pengujian, 80% untuk pelatihan
# random_state=42: memastikan pembagian data konsisten setiap kali dijalankan

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menyimpan jumlah fitur dari data input
n_features = X.shape[2]

# Menampilkan bentuk (shape) dari data latih dan data uji
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13879, 20, 4), (3470, 20, 4), (13879, 20, 4), (3470, 20, 4))

In [8]:
# Fungsi objective digunakan oleh Optuna untuk melakukan optimasi hyperparameter pada model LSTM.
# trial: objek dari Optuna yang digunakan untuk mencoba berbagai kombinasi hyperparameter.

def objective(trial):
  # Memilih jumlah unit LSTM dari beberapa pilihan
  units = trial.suggest_categorical('units', [32, 64, 128])
  # Memilih nilai dropout dari beberapa pilihan
  dropout = trial.suggest_categorical('dropout', [0.00, 0.2, 0.4])
  # Memilih learning rate dari beberapa pilihan
  learning_rate = trial.suggest_categorical('learning_rate', [0.001, 0.0005, 0.0001])
  # Memilih fungsi aktivasi dari beberapa pilihan
  activation = trial.suggest_categorical('activation', ['tanh', 'relu'])
  # Memilih ukuran batch dari beberapa pilihan
  batch_size = trial.suggest_categorical('batch_size', [32, 64])
  # Memilih jumlah epoch dari beberapa pilihan
  epochs = trial.suggest_categorical('epochs', [10])

  # Membuat model LSTM dengan hyperparameter yang dipilih
  model = Sequential()
  model.add(LSTM(units, activation=activation, input_shape=(n_in, n_features)))
  model.add(Dropout(dropout))
  model.add(RepeatVector(n_out))
  model.add(LSTM(units, activation=activation, return_sequences=True))
  model.add(Dropout(dropout))
  model.add(TimeDistributed(Dense(n_features, activation='relu')))

  # Menyusun model dengan optimizer Adam dan loss function MSE
  model.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError(), metrics=['mae'])

  # Melatih model dengan data latih dan validasi
  history = model.fit(
      X_train, y_train,
      epochs=epochs,
      batch_size=batch_size,
      validation_split=0.2,
      verbose=1
  )

  # Mengambil nilai minimum dari val_loss selama pelatihan
  val_loss = min(history.history['val_loss'])

  # Mengembalikan nilai val_loss sebagai hasil optimasi
  return val_loss

In [ ]:
# Membuat studi Optuna untuk optimasi hyperparameter dengan tujuan meminimalkan loss
study = optuna.create_study(direction='minimize')
# Melakukan optimasi dengan memanggil fungsi objective sebanyak 30 percobaan (trials)
study.optimize(objective, n_trials=30)

[I 2025-08-11 18:21:32,859] A new study created in memory with name: no-name-56202cfb-ba8a-48db-ad1e-870ec629d384
c:\Users\Muhammad Mahdi\anaconda3\envs\prediksi-polusi\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10


In [ ]:
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

Best hyperparameters:
units: 128
dropout: 0.0
learning_rate: 0.001
activation: relu
batch_size: 32
epochs: 10


In [ ]:
best_params = study.best_trial.params
# Fungsi untuk membangun model LSTM dengan hyperparameter terbaik
def build_best_model(params):
    model = Sequential()
    # Menambahkan layer LSTM pertama
    model.add(LSTM(params['units'], activation=params['activation'], input_shape=(n_in, n_features)))
    # Menambahkan layer Dropout untuk mencegah overfitting
    model.add(Dropout(params['dropout']))
    # Menambahkan RepeatVector untuk mengulang output agar sesuai dengan jumlah timestep output
    model.add(RepeatVector(n_out))
    # Menambahkan layer LSTM kedua
    model.add(LSTM(params['units'], activation=params['activation'], return_sequences=True))
    # Menambahkan layer Dropout kedua
    model.add(Dropout(params['dropout']))
    # Menambahkan TimeDistributed Dense layer untuk menghasilkan output pada setiap timestep
    model.add(TimeDistributed(Dense(n_features, activation='relu')))

    # Menggunakan optimizer Adam dengan learning rate terbaik
    optimizer = Adam(learning_rate=params['learning_rate'])
    # Menyusun model dengan loss function MeanSquaredError dan metric MAE
    model.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=['mae'])
    return model

# Membuat model dengan hyperparameter terbaik
model = build_best_model(best_params)

# Melatih model dengan data latih, menggunakan batch size terbaik dan 50 epoch
model.fit(X_train, y_train, epochs=50, batch_size=best_params['batch_size'], validation_split=0.2)

Epoch 1/50
347/347 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - loss: 0.0117 - mae: 0.0681 - val_loss: 0.0025 - val_mae: 0.0316
Epoch 2/50
347/347 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0026 - mae: 0.0326 - val_loss: 0.0024 - val_mae: 0.0307
Epoch 3/50
347/347 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0025 - mae: 0.0317 - val_loss: 0.0023 - val_mae: 0.0302
Epoch 4/50
347/347 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0025 - mae: 0.0307 - val_loss: 0.0024 - val_mae: 0.0311
Epoch 5/50
347/347 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0024 - mae: 0.0306 - val_loss: 0.0023 - val_mae: 0.0297
Epoch 6/50
347/347 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0024 - mae: 0.0302 - val_loss: 0.0022 - val_mae: 0.0284
Epoch 7/50
347/347 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0024 - mae: 0.0300 - val_loss: 0.0023 - val_mae: 0.0304
Epoch 8/50
347/347 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0023 - mae: 0.0296 - val_loss: 0.0021 - val_mae: 0.0279
Epoch 9/50
347/347 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - 

In [ ]:
model.save('best_lstm_model.h5')